In [1]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.4 MB/s eta 0:00:00


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    BatchNormalization,
    LeakyReLU,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
)
from tensorflow.keras import layers, models


from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import keras_tuner as kt

In [3]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!ls "content/drive/My Drive/"


ls: cannot access 'content/drive/My Drive/': No such file or directory


In [5]:

train_dir = "/content/drive/MyDrive/flowers/train/"
test_dir = "/content/drive/MyDrive/flowers/test/"


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image shape and batch size
image_shape = (100, 100)
batch_size = 128

# Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    validation_split=0.2  # Reserve 20% of training data for validation
)

# Only rescaling for validation and test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data with augmentation
print("Training Images:")
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

# Load validation data (20% of training data)
print("Validating Images:")
valid_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# Load test data (consider validation data as test data)
print("Test Images:")
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_shape,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Training Images:
Found 2514 images belonging to 5 classes.
Validating Images:
Found 626 images belonging to 5 classes.
Test Images:
Found 1228 images belonging to 5 classes.


In [7]:
# Extract class names from the datasets using class_indices
train_class_names = list(train_data.class_indices.keys())
valid_class_names = list(valid_data.class_indices.keys())
test_class_names = list(test_data.class_indices.keys())

# Print class names
print("Training Class Names:", train_class_names)
print("Validation Class Names:", valid_class_names)
print("Test Class Names:", test_class_names)

# Print the number of classes
print("Number of Training Classes:", len(train_class_names))
print("Number of Validation Classes:", len(valid_class_names))
print("Number of Test Classes:", len(test_class_names))

Training Class Names: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Validation Class Names: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Test Class Names: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
Number of Training Classes: 5
Number of Validation Classes: 5
Number of Test Classes: 5


In [8]:
class_names = train_class_names


In [9]:
import keras
import tensorflow as tf

model = keras.Sequential()

In [10]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        64,
        kernel_size=3,
        activation='relu',
        input_shape=(100, 100, 3)))  # Match input data shape
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Choice('layer_size', [64, 128, 256]),
        activation='relu'))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))  # Match target shape

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            ),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model


In [11]:
# Initialize the tuner

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    max_consecutive_failed_trials=5,  # Default is 3
    # directory='my_dir',
    # project_name='intro_to_kt'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
tuner.search_space_summary()

tuner.search(
    train_data,
    validation_data=valid_data,
    epochs=10,
    batch_size= 128 ,
)

best_model = tuner.get_best_models()[0]


Trial 3 Complete [00h 06m 38s]
val_accuracy: 0.5559105277061462

Best val_accuracy So Far: 0.5894568562507629
Total elapsed time: 00h 46m 44s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [14]:
best_model.save('keras_tuned.keras')

In [20]:
# Compile the Model
loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
optimizer = Adam(learning_rate=1e-4)
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
callbacks = [early_stopping, reduce_lr]

In [19]:
# Evaluate on the training data
train_loss, train_accuracy = best_model.evaluate(train_data, verbose=0)
print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

# Evaluate on the validation data
val_loss, val_accuracy = best_model.evaluate(valid_data, verbose=0)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

# Print final metrics
print("\nFinal Model Performance")
print(f"Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")
print(f"Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")


Training Loss: 0.3612, Training Accuracy: 0.9312
Validation Loss: 1.1743, Validation Accuracy: 0.5895

Final Model Performance
Training Accuracy: 0.9312, Validation Accuracy: 0.5895
Training Loss: 0.3612, Validation Loss: 1.1743


# Testing and Evaluation


In [15]:
Y_pred = best_model.predict(test_data)
score = best_model.evaluate(test_data)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 406s 45s/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 617ms/step - accuracy: 0.5047 - loss: 1.1959
Test loss: 1.1771994829177856
Test accuracy: 0.526872992515564


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# Define class names if you haven’t already
CLASS_NAMES = class_names  # Update according to your labels

# Generate predictions on the validation set
val_predictions = best_model.predict(valid_data)
val_pred_classes = np.argmax(val_predictions, axis=1)
val_true_classes = np.concatenate([y for x, y in valid_data], axis=0)  # Get true labels

# Create confusion matrix
conf_matrix = confusion_matrix(val_true_classes, val_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 649ms/step
